# Azure Landing Zones

Note: The below is for deploying to Azure global regions only. **NOT** for deploying to Azure China regions. This is due to minor difference in services which are available in Azure global and in Azure China, but the feature parity gap is narrowing.

## Connection

In [ ]:
$subscription   = ''

Connect-AzAccount -SubscriptionId $subscription

In [ ]:
Get-AzSubscription | fl *

## ALZ-Bicep Initial Setup

In [ ]:
## ALZ-BICEP SETUP
## ================================================================================

$localRepo          = "C:/Git/ALZ-Bicep"

$parLocation        = "southafricanorth"        # Deplopyment Region
$parCompanyPrefix   = "alz"                 # default="alz", min 2-chars max 10-chars
$TopLevelMGPrefix   = $parCompanyPrefix

## Management Group Parameters
## ================================================================================
$parTopLevelManagementGroupPrefix       = $parCompanyPrefix
$parTopLevelManagementGroupDisplayName  = $parCompanyPrefix + " Azure Landing Zones"
$parTopLevelManagementGroupParentId     = ""    # Intermediate root Management Group
$parLandingZoneMgAlzDefaultsEnable      = 0     # Default=True(1) - Corp and Online default Landing Zones
$parLandingZoneMgConfidentialEnable     = 0     # Default=False(0) - Confidential default Landing Zones
$parLandingZoneMgChildren               = @{}    # Custom Landing Zones - Defaults = Corp & Online
$parLandingZoneMgChildren              += @{"ALZ01" = @{'displayname'='AZL Test 01'}}
$parLandingZoneMgChildren              += @{"ALZ02" = @{'displayname'='AZL Test 02'}}

## Subscription IDs
$PlatformSubscriptionId                     = $subscription     # Platform Management Group
$ManagementSubscriptionId                   = ""                # Management Managment Grouo (Logging)
$ConnectivitySubscriptionId                 = ""                # Connectivity Management Group

if (!$ManagementSubscriptionId) { $ManagementSubscriptionId = $PlatformSubscriptionId }
if (!$ConnectivitySubscriptionId) { $ConnectivitySubscriptionId = $PlatformSubscriptionId }

## Subscription ID Placments
## NOTE: Formated as:                       = @("xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx", "yyyyyyyy-yyyy-yyyy-yyyy-yyyyyyyyyyyy")
##
$parIntRootMgSubs                           = @()               # Intermediate Root Management Group Subs
$parPlatformMgSubs                          = @()               # Platform Management Group Subs
$parPlatformManagementMgSubs                = @()               # (Platform) Management Management Group Subs
$parPlatformConnectivityMgSubs              = @()               # (Platform) Connectivity Management Group Subs
$parPlatformIdentityMgSubs                  = @()               # (Platform) Identity Management Group
$parLandingZonesMgSubs                      = @()               # Landing Zones Management Group Subs
$parLandingZonesCorpMgSubs                  = @()               # Corp (Landing Zones) Management Group Subs             
$parLandingZonesOnlineMgSubs                = @()               # Online (Landing Zones) Management Group Subs
$parLandingZonesConfidentialCorpMgSubs      = @()               # Confidential Corp (Landing Zones) Management Group Subs
$parLandingZonesConfidentialOnlineMgSubs    = @()               # Confidential Online (Landing Zones) Management Group Subs
$parDecommissionedMgSubs                    = @()               # Decommissioned Management Group Subs
$parSandboxMgSubs                           = @()               # Sandbox Management Group            

$parLandingZoneMgChildrenSubs              = @{}                # Additional child Management Groups of the Landing Zones Management Group
#$parLandingZoneMgChildrenSubs             += @{"ALZ01" = @{"subscriptions" = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx", "yyyyyyyy-yyyy-yyyy-yyyy-yyyyyyyyyyyy"}}
#$parLandingZoneMgChildrenSubs             += @{"ALZ02" = @{"subscriptions" = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx", "yyyyyyyy-yyyy-yyyy-yyyy-yyyyyyyyyyyy"}}

$parTags                                    =  @{"ALZ"      = $parCompanyPrefix;}
$parTags                                    += @{"Env"      = "Prod"}
#$parTags                                   += @{"Owner"    = "replace_me@example.com"}     

$parTelemetryOptOut                         = 0                 # Default=False (0)

#DEBUG
#Write-Host "<Test>`n"
#$parLandingZoneMgChildrenSubs
#$parLandingZoneMgChildrenSubs.gettype()
#Write-Host "`n</Test>`n"
#/DEBUG

Write-Host "`n$(Get-Date) - Initial parameters set`n"

## 1 Management Groups


Configures the management group hierarchy to support Azure Landing Zone reference implementation.  
**Prerequisites:** Owner role assignment at / root management group & ALZ-Bicep Initial Setup  
**Reference:** https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/managementGroups

In [ ]:
## MANAGEMENT GROUPS DEPLOYMENT
## ================================================================================

## Management Group Configuration Object
## ================================================================================
$inputObject = @{
  DeploymentName                        = $parCompanyPrefix + "-MGDeployment-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location                              = $parLocation
  parTopLevelManagementGroupPrefix      = $parTopLevelManagementGroupPrefix
  parTopLevelManagementGroupDisplayName = $parTopLevelManagementGroupDisplayName
  parTopLevelManagementGroupParentId    = $parTopLevelManagementGroupParentId
  parLandingZoneMgAlzDefaultsEnable     = $parLandingZoneMgAlzDefaultsEnable
  parLandingZoneMgConfidentialEnable    = $parLandingZoneMgConfidentialEnable
  parLandingZoneMgChildren              = $parLandingZoneMgChildren
  TemplateFile                          = "$localRepo/infra-as-code/bicep/modules/managementGroups/managementGroups.bicep"
  TemplateParameterFile                 = "$localRepo/infra-as-code/bicep/modules/managementGroups/parameters/managementGroups.parameters.all.json"
}

## Management Group TEST Deployment
## ================================================================================
$inputObject
write-host `n
New-AzTenantDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Management Group Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzTenantDeployment @inputObject -Verbose

## 2 Custom Policy Definitions	

Configures Custom Policy Definitions at the organization management group.  
**Prerequisites:** Management Groups  
**Reference:**  https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/policy/definitions

In [ ]:
## CUSTOM POLICY DEFINITIONS
## ================================================================================

## Custom Policy Definitions Parameters
## ================================================================================
$parTargetManagementGroupId = $parCompanyPrefix

## Custom Policy Definitions Configuration Object
## ================================================================================
$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "-PolicyDefsDeployment-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location              = $parLocation
  ManagementGroupId     = $parCompanyPrefix
  TemplateFile          = "$localRepo\infra-as-code/bicep/modules/policy/definitions/customPolicyDefinitions.bicep"
  TemplateParameterFile = "$localRepo\infra-as-code/bicep/modules/policy/definitions/parameters/customPolicyDefinitions.parameters.all.json"  
}
if ($parTargetManagementGroupId) { $inputObject["parTargetManagementGroupId"] = $parTargetManagementGroupId }

## Custom Policy Definitions TEST Deployment
## ================================================================================
$inputObject
write-host `n
New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Custom Policy Definitions Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzManagementGroupDeployment @inputObject -Verbose

## 3 Custom RBAC Role Definitions

Configures custom roles based on Cloud Adoption Framework's recommendations at the organization management group.  
**Prerequisites:** Management Groups.  
**Reference:** https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/customRoleDefinitions 

In [ ]:
## CUSTOM ROLE DEFINITIONS
## ================================================================================

## Custom Role Definitions Parameters
## ================================================================================
$parAssignableScopeManagementGroupId = $parCompanyPrefix

## Custom Role Definitions Configuration Object
## ================================================================================
$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "-CustomRoleDefsDeployment-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location              = $parLocation
  ManagementGroupId     = $parCompanyPrefix
  TemplateFile          = "$localRepo\infra-as-code/bicep/modules/customRoleDefinitions/customRoleDefinitions.bicep"
  TemplateParameterFile = "$localRepo\infra-as-code/bicep/modules/customRoleDefinitions/parameters/customRoleDefinitions.parameters.all.json"
}
if ($parAssignableScopeManagementGroupId) { $inputObject["parAssignableScopeManagementGroupId"] = $parAssignableScopeManagementGroupId }

## Custom Role Definitions TEST Deployment
## ================================================================================
New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Custom Role Definitions Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzManagementGroupDeployment @inputObject -Verbose

## 4 Logging & Security

Configures a centrally managed Log Analytics Workspace, Automation Account and Sentinel in the Logging subscription.  
**Prerequisites:** Management Groups & Subscription for Log Analytics and Sentinel.  
**Reference:** https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/logging

In [ ]:
## LOG ANALYTICS AND SENTINEL
## ================================================================================

## Log Analytics and Sentinel Parameters
## ================================================================================

$parLogAnalyticsWorkspaceName               = $parCompanyPrefix + "-log-analytics"
$parLogAnalyticsWorkspaceLocation           = $parLocation
$parLogAnalyticsWorkspaceSkuName            = "PerGB2018"     # Default value: PerGB2018
                                                              # Allowed values: PerGB2018, CapacityReservation, LACluster (also legacy Skus: Free, PerNode, Premium, Standalone, Standard)
$parLogAnalyticsWorkspaceLogRetentionInDays = "30"            # Default value: 365
$parLogAnalyticsWorkspaceSolutions          = @('AgentHealthAssessment', 'AntiMalware', 'ChangeTracking', 'Security', 'SecurityInsights', 'ServiceMap', 'SQLAdvancedThreatProtection', 'SQLVulnerabilityAssessment', 'SQLAssessment', 'Updates', 'VMInsights')

$parAutomationAccountName                   = $parCompanyPrefix + "-automation-account"
$parAutomationAccountLocation               = $parLocation

# Set Platform management subscripion ID
if (!$ManagementSubscriptionId) { $ManagementSubscriptionId = $PlatformSubscriptionId }

$myResourceGroupName    = "rg-$parCompanyPrefix-logging-001"

## Log Analytics and Sentinel Configuration Object
## ================================================================================

$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "-LoggingDeploy-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  ResourceGroupName     = $myResourceGroupName
  TemplateFile          = "$localRepo/infra-as-code/bicep/modules/logging/logging.bicep"
  TemplateParameterFile = "$localRepo/infra-as-code/bicep/modules/logging/parameters/logging.parameters.all.json"
  parTags               = $parTags 
}

if ($parLogAnalyticsWorkspaceName) { $inputObject["parLogAnalyticsWorkspaceName"] = $parLogAnalyticsWorkspaceName }
if ($parLogAnalyticsWorkspaceLocation) { $inputObject["parLogAnalyticsWorkspaceLocation"] = $parLogAnalyticsWorkspaceLocation }
if ($parLogAnalyticsWorkspaceSkuName) { $inputObject["parLogAnalyticsWorkspaceSkuName"] = $parLogAnalyticsWorkspaceSkuName }
if ($parLogAnalyticsWorkspaceLogRetentionInDays) { $inputObject["parLogAnalyticsWorkspaceLogRetentionInDays"] = $parLogAnalyticsWorkspaceLogRetentionInDays }
if ($parLogAnalyticsWorkspaceSolutions) { $inputObject["parLogAnalyticsWorkspaceSolutions"] = $parLogAnalyticsWorkspaceSolutions }
if ($parAutomationAccountName) { $inputObject["parAutomationAccountName"] = $parAutomationAccountName }
if ($parAutomationAccountLocation) { $inputObject["parAutomationAccountLocation"] = $parAutomationAccountLocation }

Select-AzSubscription -SubscriptionId $ManagementSubscriptionId -Force 

if (!(Get-AzResourceGroup $myResourceGroupName -ErrorAction SilentlyContinue))
{ 
  try 
  {
    Write-Host "`nCreating the $myResourceGroupName resource group`n"
    New-AzResourceGroup -Name $myResourceGroupName -Location $parLocation -ErrorAction SilentlyContinue
  } 
  catch 
  { 
    Write-Host "`nThere was an error creaating the $myResourceGroupName resource group`n" -ForegroundColor Red
    Break 
  }
}
else 
{
  Write-Host "`nThe $myResourceGroupName resource group already exists`n"
}

## Log Analytics and Sentinel TEST Deployment
## NOTE: The resource group will be created before the -WhatIf testing
## ================================================================================
New-AzResourceGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
$ManagementSubscriptionId

In [ ]:
## Log Analytics and Sentinel Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected

New-AzResourceGroupDeployment @inputObject -Verbose

# Retrieve for Management Groups Diagnostics Module
$parLogAnalyticsWorkspaceResourceId = (Get-AzResourceGroupDeployment -ResourceGroupName $myResourceGroupName -Name $inputObject["DeploymentName"]).Outputs.outLogAnalyticsWorkspaceId.value

### 4.1 Management Groups Diagnostic Settings  

Enable Diagnostic Settings for management Groups to the Log Analytics Workspace created in the Logging subscription.  
**Prerequisites:** Management Groups & Log Analytics Workspace.  
https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/orchestration/mgDiagSettingsAll 

In [ ]:
## MANAGEMENT GROUPS DIAGNOSTICS SETTING
## ================================================================================

## Management Groups Diagnostics Parameters
## ================================================================================

# Set in Management Group Deployment Module
# $parTopLevelManagementGroupPrefix       = $parCompanyPrefix
# $parLandingZoneMgAlzDefaultsEnable      = 0 
# $parLandingZoneMgConfidentialEnable     = 0

# Created in Log Analytics and Sentinel Module
# $parLogAnalyticsWorkspaceResourceId = (Get-AzResourceGroupDeployment -ResourceGroupName $myResourceGroupName -Name $inputObject["DeploymentName"]).Outputs.outLogAnalyticsWorkspaceId.value

$parLandingZoneMgChildren               = "ALZ01","ALZ02"   #NOTE: Automate this from Management Groups Definition

## Management Groups Diagnostics Object
## ================================================================================

$inputObject = @{
  DeploymentName                      = $parCompanyPrefix + "-mgDiagSettingsAll-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location                            = $parLocation
  ManagementGroupId                   = $parCompanyPrefix
  parTopLevelManagementGroupPrefix    = $parTopLevelManagementGroupPrefix      
  parLandingZoneMgAlzDefaultsEnable   = $parLandingZoneMgAlzDefaultsEnable    
  parLandingZoneMgConfidentialEnable  = $parLandingZoneMgConfidentialEnable  
  parLogAnalyticsWorkspaceResourceId  = $parLogAnalyticsWorkspaceResourceId
  TemplateFile          = "$localRepo\infra-as-code/bicep/orchestration/mgDiagSettingsAll/mgDiagSettingsAll.bicep"
  TemplateParameterFile = "$localRepo\infra-as-code/bicep/orchestration/mgDiagSettingsAll/parameters/mgDiagSettingsAll.parameters.all.json"
}

if ($parLandingZoneMgChildren) { $inputObject["parLandingZoneMgChildren"] = $parLandingZoneMgChildren }

## Management Group Diagnostic Setting TEST Deployment
## ================================================================================
$inputObject
write-host `n
New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf -DeploymentDebugLogLevel All

In [ ]:
## Management Groups Diagnostic Settings Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzManagementGroupDeployment @inputObject -Verbose

## 5 Hub Networking


Azure supports two types of hub-and-spoke design, VNet hub and Virtual WAN hub. Creates resources in the Connectivity subscription.  
**Prerequisites:** Management Groups, Subscription for Hub Networking.  
https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/hubNetworking 

### 5.1 Hub and Spoke VNet


Creates Hub networking infrastructure with Azure Firewall to support Hub & Spoke network topology in the Connectivity subscription.  
**Prerequisites:** Management Groups, Subscription for Hub Networking.  
https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/hubNetworking

Can deploy the following resources:  
1. Virtual Network (VNet)
2. Subnets
3. VPN Gateway/ExpressRoute Gateway
4. Azure Firewall
5. Azure Firewall Policies
6. Private DNS Zones
7. DDoS Network Protection Plan
8. Bastion
9. Route Table

In [ ]:
## HUB NETWORKING - HUB AND SPOKE VNET
## ================================================================================

## Parameters
## --------------------------------------------------------------------------------
$parHubNetworkName                  = "vnet-" + $parCompanyPrefix + "-hub-" + $parLocation 
$parHubNetworkAddressPrefix         = "10.20.0.0/16"

## Subnet Parameters
## --------------------------------------------------------------------------------
$AzureBastionSubnet                 = "10.20.15.0/24"
$GatewaySubnet                      = "10.20.252.0/24"
$AzureFirewallSubnet                = "10.20.254.0/24"

## Bastion Host Parameters
## --------------------------------------------------------------------------------
$parAzBastionEnabled                = 0   # Default=True (1)
#$parAzBastionName                  = $parCompanyPrefix + "-bastion"
#$parAzBastionSku                   = "Standard"
#$parAzBastionNsgName               = "nsg-AzureBastionSubnet"

## DDoS Network Protection Parameters
## --------------------------------------------------------------------------------
$parDdosEnabled                     = 0   # Default=True (1)
#$parDdosPlanName                   = $parCompanyPrefix + "ddos-plan"

## Azure Firewall Parameters
## --------------------------------------------------------------------------------
$parAzFirewallEnabled               = 0   # Default=True (1)
#$parAzFirewallName                 = "alz-azfw-eastus"
#$parAzFirewallPoliciesName         = "alz-azfwpolicy-eastus"
#$parAzFirewallTier                 = "Standard"
#$parAzFirewallAvailabilityZones    = ""
#$parAzFirewallDnsProxyEnabled      = 0   # Default=True (1)

## TO DO
#parAzFirewallAvailabilityZones     = ""
#parAzErGatewayAvailabilityZones    = ""
#parAzVpnGatewayAvailabilityZones   = ""

## VPN Gateway Parameters
## --------------------------------------------------------------------------------
## NOTE: See note below :/
$parVpnGatewayConfig = @{ name                                = $parCompanyPrefix + "-Vpn-Gateway"; `
                    	    gatewayType                         = "Vpn"; `
                          sku                                 = "VpnGw1"; `
                          vpnType                             = "RouteBased"; `
                          generation                          = "Generation1"; `
                          enableBgp                           = 0; #False `
                          activeActive                        = 0; #False `
                          enableBgpRouteTranslationForNat     = 0; #False `
                          enableDnsForwarding                 = 0; #False `
                          asn                                 = "65515"; `
                          bgpPeeringAddress                   = ""; `
                          bgpsettings = @{ asn 			      = "65515"; `
                                    	   bgpPeeringAddress  = ""; `
                                    	   peerWeight         = "5" 
                          }
}  
## NOTE: Comment out this line if VPN Gateway *is* required
$parVpnGatewayConfig = @{ 'name'    = "noconfigVpn" } 
## /NOTE

## ExpressRoute Gateway Parameters
## --------------------------------------------------------------------------------
## NOTE: See note below :/
$parExpressRouteGatewayConfig     =  @{ name                                = $parCompanyPrefix + "-ExpressRoute-Gateway"; `
                                    gatewayType                         = "ExpressRoute"; `
                                    sku                                 = "Standard"; `
                                    vpnType                             = "RouteBased"; `
                                    generation                          = "None"; `
                                    enableBgp                           = 0; #False `
                                    activeActive                        = 0; #False `
                                    enableBgpRouteTranslationForNat     = 0; #False `
                                    enableDnsForwarding                 = 0; #False `
                                    asn                                 = "65515"; `
                                    bgpPeeringAddress                   = ""; `
                                    bgpsettings = @{ asn 			    = "65515"; `
                                    				 bgpPeeringAddress  = ""; `
                                    				 peerWeight         = "5" 
                                    }
}
## NOTE: Comment out this line if ER Gateway *IS* required
$parExpressRouteGatewayConfig     = @{ 'name' = "noconfigEr" }
## /NOTE




$parHubRouteTableName = "alz-hub-routetable"
$parDisableBgpRoutePropagation = 0      # Default=False (0)

$parPrivateDnsZonesEnabled  = 1   # Default=True (1)

## Other Parameters
## --------------------------------------------------------------------------------




## TO DO
# $parDnsServerIps
# parPublicIpSku


Select-AzSubscription -SubscriptionId $ConnectivitySubscriptionId

$TopLevelMGPrefix     = $parCompanyPrefix
$myResourceGroupName  = "rg-$TopLevelMGPrefix-hub-networking-001"

## NOTE: The DNS Zone privatelink.{dnsPrefix}.database.windows.net is not deployed 
## by default as the DNS Prefix is individual.
$parPrivateDnsZones = @(
      "privatelink.$parLocation.azmk8s.io",
      "privatelink.$parLocation.batch.azure.com",
      "privatelink.$parLocation.kusto.windows.net",
        "privatelink.adf.azure.com",
        "privatelink.afs.azure.net",
        "privatelink.agentsvc.azure-automation.net",
        "privatelink.analysis.windows.net",
        "privatelink.api.azureml.ms",
        "privatelink.azconfig.io",
        "privatelink.azure-api.net",
        "privatelink.azure-automation.net",
        "privatelink.azurecr.io",
        "privatelink.azure-devices.net",
        "privatelink.azure-devices-provisioning.net",
        "privatelink.azurehdinsight.net",
        "privatelink.azurehealthcareapis.com",
        "privatelink.azurestaticapps.net",
        "privatelink.azuresynapse.net",
        "privatelink.azurewebsites.net",
        "privatelink.batch.azure.com",
        "privatelink.blob.core.windows.net",
        "privatelink.cassandra.cosmos.azure.com",
        "privatelink.cognitiveservices.azure.com",
        "privatelink.database.windows.net",
        "privatelink.datafactory.azure.net",
        "privatelink.dev.azuresynapse.net",
        "privatelink.dfs.core.windows.net",
        "privatelink.dicom.azurehealthcareapis.com",
        "privatelink.digitaltwins.azure.net",
        "privatelink.directline.botframework.com",
        "privatelink.documents.azure.com",
        "privatelink.eventgrid.azure.net",
        "privatelink.file.core.windows.net",
        "privatelink.gremlin.cosmos.azure.com",
        "privatelink.guestconfiguration.azure.com",
        "privatelink.his.arc.azure.com",
        "privatelink.kubernetesconfiguration.azure.com",
        "privatelink.managedhsm.azure.net",
        "privatelink.mariadb.database.azure.com",
        "privatelink.media.azure.net",
        "privatelink.mongo.cosmos.azure.com",
        "privatelink.monitor.azure.com",
        "privatelink.mysql.database.azure.com",
        "privatelink.notebooks.azure.net",
        "privatelink.ods.opinsights.azure.com",
        "privatelink.oms.opinsights.azure.com",
        "privatelink.pbidedicated.windows.net",
        "privatelink.postgres.database.azure.com",
        "privatelink.prod.migration.windowsazure.com",
        "privatelink.purview.azure.com",
        "privatelink.purviewstudio.azure.com",
        "privatelink.queue.core.windows.net",
        "privatelink.redis.cache.windows.net",
        "privatelink.redisenterprise.cache.azure.net",
        "privatelink.search.windows.net",
        "privatelink.service.signalr.net",
        "privatelink.servicebus.windows.net",
        "privatelink.siterecovery.windowsazure.com",
        "privatelink.sql.azuresynapse.net",
        "privatelink.table.core.windows.net",
        "privatelink.table.cosmos.azure.com",
        "privatelink.tip1.powerquery.microsoft.com",
        "privatelink.token.botframework.com",
        "privatelink.vaultcore.azure.net",
        "privatelink.web.core.windows.net",
        "privatelink.webpubsub.azure.com"
    )







## Hub Networking Configuration Object
## ================================================================================
$inputObject = @{
  parLocation                 = $parLocation
  parHubNetworkName           = $parHubNetworkName
  parHubNetworkAddressPrefix  = $parHubNetworkAddressPrefix

  
  DeploymentName        = $parCompanyPrefix + "-HubNetworkingDeploy-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  ResourceGroupName     = $myResourceGroupName
  parPrivateDnsZones    = $parPrivateDnsZones
  TemplateFile          = "$localRepo/infra-as-code/bicep/modules/hubNetworking/hubNetworking.bicep"
  TemplateParameterFile = "$localRepo/infra-as-code/bicep/modules/hubNetworking/parameters/hubNetworking.parameters.all.json"
}

$inputObject["parSubnets"] = @{ Name = "AzureBastionSubnet"; ipAddressRange = "$AzureBastionSubnet"}, `
                             @{ Name = "GatewaySubnet"; ipAddressRange = "$GatewaySubnet"}, `
                             @{ Name = "AzureFirewallSubnet"; ipAddressRange = "$AzureFirewallSubnet"}

# if ($AzureBastionSubnet) { $inputObject["parSubnets"] = @{ 'name' = "AzureBastionSubnet", 'ipAddressRange' = "$AzureBastionSubnet" } }

$inputObject["parVpnGatewayConfig"] = $parVpnGatewayConfig 
$inputObject["parExpressRouteGatewayConfig"] = $parExpressRouteGatewayConfig 

if (!(Get-AzResourceGroup $myResourceGroupName -ErrorAction SilentlyContinue))
{ 
  try 
  {
    Write-Host "`nCreating the $myResourceGroupName resource group"
    New-AzResourceGroup -Name $myResourceGroupName -Location $parLocation -ErrorAction SilentlyContinue
  }
  catch 
  {
    Write-Host "`nThere was an error creaating the $myResourceGroupName resource group" -ForegroundColor Red
    Break
  }
}
else
{
  Write-Host "`nThe $myResourceGroupName resource group already exists"
}

## Hub Networking TEST Deployment
## ================================================================================

# DEBUG
$inputObject
write-host `n
# /DEBUG

New-AzResourceGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Hub Networking Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzResourceGroupDeployment @inputObject -Verbose

## Outputs
$parDdosProtectionPlanId = (Get-AzResourceGroupDeployment -ResourceGroupName $myResourceGroupName -Name $inputObject["DeploymentName"]).Outputs.parDdosProtectionPlanId.value
$outHubVirtualNetworkId = (Get-AzResourceGroupDeployment -ResourceGroupName $myResourceGroupName -Name $inputObject["DeploymentName"]).Outputs.outHubVirtualNetworkId.value

### 5.2 Virtual WAN (TO DO)


Deploys the following resources:  
1. Virtual WAN
1. Virtual Hub. *The virtual hub is a prerequisite to connect to either a VPN Gateway, an ExpressRoute Gateway or an Azure Firewall to the virtual WAN*
1. VPN Gateway
1. ExpressRoute Gateway
1. Azure Firewall
1. Azure Firewall policy
1. DDoS Network Protection Plan
1. Private DNS Zones

In [ ]:
## HUB NETWORKING - Virtual WAN (vWAN)
## ================================================================================

## ================================================================================
## NOTE: Run Hub Networking Setup
## ================================================================================

## Parameters
## --------------------------------------------------------------------------------
$parVirtualHubAddressPrefix = "10.100.0.0/23"

$parAzFirewallTier = "Standard"
$parVirtualHubEnabled = 1
$parVpnGatewayEnabled = 1
$parExpressRouteGatewayEnabled = 1
$parAzFirewallEnabled = 1
$parAzFirewallDnsProxyEnabled = 1

$parVirtualWanName = $parCompanyPrefix + "-vwan-" + $parLocation
$parVirtualWanHubName = $parCompanyPrefix + "-vwan-" + $parLocation

$parVpnGatewayName = $parCompanyPrefix + "-vpngw-" + $parLocation
$parExpressRouteGatewayName = $parCompanyPrefix + "-ergw-" + $parLocation
$parAzFirewallName = $parCompanyPrefix + "-fw-" + $parLocation

$parAzFirewallAvailabilityZones = @{}

$parAzFirewallPoliciesName = $parCompanyPrefix + "-azfwpolicy-" + $parLocation

$parVpnGatewayScaleUnit = 1
$parExpressRouteGatewayScaleUnit = 1
$parDdosEnabled = 1
$parDdosPlanName = $parCompanyPrefix + "-azfwpolicy-" + $parLocation

$parPrivateDnsZonesEnabled = 1
$parPrivateDnsZones = $parPrivateDnsZones

### TO SORT
$parVirtualNetworkIdToLink": {
      "value": "/subscriptions/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx/resourceGroups/HUB_Networking_POC/providers/Microsoft.Network/virtualNetworks/alz-hub-eastus"


In [ ]:
if (!$ConnectivitySubscriptionId) { $ConnectivitySubscriptionId = $PlatformSubscriptionId }

Select-AzSubscription -SubscriptionId $ConnectivitySubscriptionId

$TopLevelMGPrefix     = $parCompanyPrefix
$myResourceGroupName  = "rg-$TopLevelMGPrefix-hub-networking-001"

# Parameters necessary for deployment
$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "-vwanConnectivityDeploy-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  ResourceGroupName     = $myResourceGroupName
  TemplateFile          = "$localRepo/infra-as-code/bicep/modules/vwanConnectivity/vwanConnectivity.bicep"
  TemplateParameterFile = "$localRepo/infra-as-code/bicep/modules/vwanConnectivity/parameters/vwanConnectivity.parameters.all.json"
}

if (!(Get-AzResourceGroup $myResourceGroupName -ErrorAction SilentlyContinue))
{ 
  try 
  {
    Write-Host "`nCreating the $myResourceGroupName resource group"
    New-AzResourceGroup -Name $myResourceGroupName -Location $parLocation -ErrorAction SilentlyContinue
  }
  catch 
  {
    Write-Host "`nThere was an error creaating the $myResourceGroupName resource group" -ForegroundColor Red
    Break
  }
}
else
{
  Write-Host "`nThe $myResourceGroupName resource group already exists"
}

$inputObject
write-host `n

# Test Deploy
New-AzResourceGroupDeployment @inputObject -Verbose -WhatIf

## 6 RBAC Role Assignments (TO DO)

Creates role assignments using built-in and custom role definitions.  
**Prerequisites:** Management Groups & Subscriptions.  
https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/modules/roleAssignments

Role assignments can be performed for:
- Managed Identities (System and User Assigned)
- Service Principals
- Security Groups

In [ ]:
# Identify Object Id for User Assigned / System Assigned Managed Identity
# Example: (Get-AzADServicePrincipal -DisplayName 'alz-managed-identity').Id

(Get-AzADServicePrincipal -DisplayName '<IDENTITY_NAME>').Id

In [ ]:
# Identify Object Id for Service Principal (App Registration)
# Require read permission to query Azure Active Directory
# Example:  (Get-AzADServicePrincipal -DisplayName 'Azure Landing Zone SPN').Id

(Get-AzADServicePrincipal -DisplayName '<APP_REGISTRATION_DISPLAY_NAME>').Id

In [ ]:
# Identify Object Id for Security Group
# Require read permission to query Azure Active Directory
# Example: Get-AzureADGroup -SearchString 'SG_ALZ_SECURITY'

Connect-AzureAD
(Get-AzureADGroup -SearchString '<SECURITY_GROUP_NAME>').ObjectId

### 6.1 Assignment to Management Groups (In Progress)

In [ ]:
# roleAssignmentManagementGroup.managedIdentity

$parRoleAssignmentNameGuid =  "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
$parRoleDefinitionId = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

$parAssigneePrincipalType = "ServicePrincipal"   # managedIdentity and servicePrincipal
#$parAssigneePrincipalType = "Group"             # Security Groups

$parAssigneeObjectId = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
## Note: Object ID of groups, service principals or managed identities. For managed identities use the principal id. For service principals, use the object ID and not the app ID

In [ ]:
  "parameters": {
    "parManagementGroupIds": {
      "value": [
        "alz-platform-connectivity",
        "alz-platform-identity"
      ]
    },

parameters": {
    "parResourceGroupIds": {
      "value": [
        "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx/xxxxxxx",
        "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx/xxxxxxx"
      ]
    }




      "parameters": {
    "parSubscriptionIds": {
      "value": [
        "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
        "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
      ]
    },

In [ ]:
$inputObject = @{
  DeploymentName        = 'alz-RoleAssignmentsDeployment-{0}' -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location              = 'eastus'
  ManagementGroupId     = 'alz'
  TemplateFile          = "infra-as-code/bicep/modules/roleAssignments/roleAssignmentManagementGroup.bicep"
  TemplateParameterFile = 'infra-as-code/bicep/modules/roleAssignments/parameters/roleAssignmentManagementGroup.servicePrincipal.parameters.all.json'
}

New-AzManagementGroupDeployment @inputObject

### 6.2 Assignment to Subscriptions

### 6.2 Assignment to Resource Groups

In [ ]:
# Identify Object Id for User Assigned / System Assigned Managed Identity
# Example: (Get-AzADServicePrincipal -DisplayName 'alz-managed-identity').Id
(Get-AzADServicePrincipal -DisplayName '<IDENTITY_NAME>').Id

# Identify Object Id for Service Principal (App Registration)
# Require read permission to query Azure Active Directory
# Example:  (Get-AzADServicePrincipal -DisplayName 'Azure Landing Zone SPN').Id
(Get-AzADServicePrincipal -DisplayName '<APP_REGISTRATION_DISPLAY_NAME>').Id

# Identify Object Id for Security Group
# Require read permission to query Azure Active Directory
# Example: Get-AzureADGroup -SearchString 'SG_ALZ_SECURITY'
Connect-AzureAD
(Get-AzureADGroup -SearchString '<SECURITY_GROUP_NAME>').ObjectId

In [ ]:
//
// Baseline deployment sample
//

// Use this sample to deploy the minimum resource configuration.

targetScope = 'managementGroup'

// ----------
// PARAMETERS
// ----------
var roleDefinitionId = '/providers/Microsoft.Authorization/roleDefinitions/8e3af657-a8ff-443c-a75c-2fe8c4bcb635'
var assigneeObjectId = '00000000-0000-0000-0000-000000000000'
// ---------
// RESOURCES
// ---------

@description('Baseline resource configuration.')
module baseline_ra '../roleAssignmentManagementGroup.bicep' = {
  name: 'baseline_ra'
  params: {
    parRoleDefinitionId: roleDefinitionId
    parAssigneePrincipalType: 'Group'
    parAssigneeObjectId: assigneeObjectId
    parTelemetryOptOut: true
    parRoleAssignmentNameGuid: guid(managementGroup().name, roleDefinitionId, assigneeObjectId)
  }
}

In [ ]:
## ROLE ASSIGNMENTS
## ================================================================================

## Parameters
## --------------------------------------------------------------------------------

$parRoleAssignmentNameGuid = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
$parRoleDefinitionId = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
$parAssigneePrincipalType = "ServicePrincipal"          #Allowed values are 'Group' (Security Group) or 'ServicePrincipal' (Service Principal or System/User Assigned Managed Identity)
$parAssigneeObjectId = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

In [ ]:
## ROLE ASSIGNMENTS
## ================================================================================
## Management Group

## Role Assignment Config Object
## ================================================================================
$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "alz-RoleAssignmentsDeployment-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location              = $parLocation
  ManagementGroupId     = $parCompanyPrefix
  TemplateFile          = "$localRepo\infra-as-code/bicep/modules/roleAssignments/roleAssignmentManagementGroup.bicep"
  TemplateParameterFile = "$localRepo\infra-as-code/bicep/modules/roleAssignments/parameters/roleAssignmentManagementGroup.servicePrincipal.parameters.all.json"
}


New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf

## 7 Subscription Placement

Moves one or more subscriptions (based on IDs) to the target Management Groups in your ALZ hierarchy.  
**Prerequisites:** Management Groups & Subscriptions.   
https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/orchestration/subPlacementAll

In [ ]:
## SUBSCRRIPTION PLACEMENT
## ================================================================================
## Subscription info 

$inputObject = @{
  DeploymentName                          = $parCompanyPrefix + "-SubPlacementAll-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location                                = $parLocation
  ManagementGroupId                       = $parCompanyPrefix
  TemplateFile                            = "$localRepo\infra-as-code/bicep/orchestration/subPlacementAll/subPlacementAll.bicep"
  TemplateParameterFile                   = "$localRepo\infra-as-code/bicep/orchestration/subPlacementAll/parameters/subPlacementAll.parameters.all.json"
  parTopLevelManagementGroupPrefix        = $parCompanyPrefix
  parIntRootMgSubs                        = $parIntRootMgSubs
  parPlatformMgSubs                       = $parPlatformMgSubs
  parPlatformManagementMgSubs             = $parPlatformManagementMgSubs
  parPlatformConnectivityMgSubs           = $parPlatformConnectivityMgSubs
  parPlatformIdentityMgSubs               = $parPlatformIdentityMgSubs
  parLandingZonesMgSubs                   = $parLandingZonesMgSubs
  parLandingZonesCorpMgSubs               = $parLandingZonesCorpMgSubs
  parLandingZonesOnlineMgSubs             = $parLandingZonesOnlineMgSubs
  parLandingZonesConfidentialCorpMgSubs   = $parLandingZonesConfidentialCorpMgSubs
  parLandingZonesConfidentialOnlineMgSubs = $parLandingZonesConfidentialOnlineMgSubs
  parLandingZoneMgChildrenSubs            = $parLandingZoneMgChildrenSubs
  parDecommissionedMgSubs                 = $parDecommissionedMgSubs
  parSandboxMgSubs                        = $parSandboxMgSubs
}

# DEBUG
# $inputObject["parLandingZoneMgChildrenSubs"]
# Write-Host "`n"
# /DEBUG

## Test Deployment
New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Deploy
New-AzManagementGroupDeployment @inputObject -Verbose

## 8 Built-In and Custom Policy Assignments

Creates policy assignments to provide governance at scale.  
**Prerequisites:** Management Groups, Log Analytics Workspace & Custom Policy Definitions 

In [ ]:
## Built-In and Custom Policy Assignments Deployment
## ================================================================================

$parMsDefenderForCloudEmailSecurityContact = "security_contact@example.com"

$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "-alzPolicyAssignmentDefaultsDeployment-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location              = $parLocation
  ManagementGroupId     = $parCompanyPrefix
  parLogAnalyticsWorkspaceResourceId  = $parLogAnalyticsWorkspaceResourceId
  TemplateFile          = "$localRepo\infra-as-code/bicep/modules/policy/assignments/alzDefaults/alzDefaultPolicyAssignments.bicep"
  TemplateParameterFile = "$localRepo\infra-as-code/bicep/modules/policy/assignments/alzDefaults/parameters/alzDefaultPolicyAssignments.parameters.all.json"
}

if ($parLogAnalyticsWorkspaceLogRetentionInDays) { $inputObject["parLogAnalyticsWorkspaceLogRetentionInDays"] = $parLogAnalyticsWorkspaceLogRetentionInDays }
if ($parAutomationAccountName) { $inputObject["parAutomationAccountName"] = $parAutomationAccountName }
if ($parMsDefenderForCloudEmailSecurityContact) { $inputObject["parMsDefenderForCloudEmailSecurityContact"] = $parMsDefenderForCloudEmailSecurityContact }
if ($parDdosProtectionPlanId) { $inputObject["parDdosProtectionPlanId"] = $parDdosProtectionPlanId }

New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Built-In and Custom Policy Assignments Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzResourceGroupDeployment @inputObject -Verbose

## 9 Spoke Networking

Creates Spoke networking infrastructure for workloads with Virtual Network Peering (optional) to support Hub & Spoke network topology or Virtual Hub Connection (optional).  
**Prerequisites:** Management Groups, Hub Networking & Subscription for spoke networking  
https://github.com/Azure/ALZ-Bicep/tree/main/infra-as-code/bicep/orchestration/hubPeeredSpoke

In [ ]:
## SPOKE NETWORKING
## ================================================================================

## Parameters
## --------------------------------------------------------------------------------
$parLocation                            = $parLocation
$parTopLevelManagementGroupPrefix       = $parCompanyPrefix

$parPeeredVnetSubscriptionId            = $ConnectivitySubscriptionId
$parPeeredVnetSubscriptionMgPlacement   = $parCompanyPrefix + "-platform-connectivity"

$parSpokeNetworkName                    = "vnet-spoke"
$parSpokeNetworkAddressPrefix           = "10.11.0.0/16"
$parDnsServerIps                        = @()
$parNextHopIpAddress                    = ""
$parDisableBgpRoutePropagation          = 0

$parSpoketoHubRouteTableName            = "rtb-spoke-to-hub"
$parHubVirtualNetworkId                 = $outHubVirtualNetworkId

$parAllowSpokeForwardedTraffic          = 0
$parAllowHubVPNGatewayTransit           = 1

In [ ]:
$inputObject = @{
  DeploymentName        = $parCompanyPrefix + "HubPeeredSpoke-{0}" -f (-join (Get-Date -Format 'yyyyMMddTHHMMssffffZ')[0..63])
  Location              = $parLocation
  ManagementGroupId     = $parCompanyPrefix
  TemplateFile          = "$localRepo\infra-as-code/bicep/orchestration/hubPeeredSpoke/hubPeeredSpoke.bicep"
  TemplateParameterFile = "$localRepo\infra-as-code/bicep/orchestration/hubPeeredSpoke/parameters/hubPeeredSpoke.parameters.all.json"
}

if ($parDdosProtectionPlanId) { $inputObject["parDdosProtectionPlanId"] = $parDdosProtectionPlanId }

## Spoke Networking TEST Deployment
## ================================================================================
$inputObject
write-host `n

New-AzManagementGroupDeployment @inputObject -Verbose -WhatIf

In [ ]:
## Spoke Networking Deployment *THIS IS NOT A TEST*
## ================================================================================
## NOTE: Test the deployment above and check the output is as expected
##
New-AzManagementGroupDeployment @inputObject -Verbose